In [2]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split

In [4]:
def load_data():
    # loading and preprocessing dataset 
    data = pd.read_csv("../dataset/annotations/annotations.csv")
    data = data[['id','image','score']]
        
    images = []
    for i in range(0,len(data)):
        filepath = data['image'][i]
        image = cv2.imread('../dataset/images/{0}'.format(filepath),cv2.IMREAD_COLOR)

        if image is not None:
            new_dimensions = (224, 224)
            resized_image = cv2.resize(image, new_dimensions, interpolation=cv2.INTER_AREA)
            images.append(resized_image)

    return np.array(images), np.array(data['score'])

In [5]:
images, ratings = load_data()
images = images / 255.0  # normalize images

print(images.shape)

X_train, X_test, y_train, y_test = train_test_split(images, ratings, test_size=0.2, random_state=42)

print(y_train)


(1985, 224, 224, 3)
[64. 79. 39. ... 71. 54. 57.]


In [6]:
base_model = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(224,224,3)
    )

# Add custom top layers for regression
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)  # Output layer for regression

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)


Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - loss: 1916.4504 - mae: 36.8307 - val_loss: 179.7826 - val_mae: 11.1879
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - loss: 203.6720 - mae: 11.8917 - val_loss: 170.3563 - val_mae: 11.3801
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - loss: 198.5348 - mae: 12.1490 - val_loss: 167.7551 - val_mae: 11.3218
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - loss: 180.8787 - mae: 11.5407 - val_loss: 168.5480 - val_mae: 11.1686
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - loss: 183.6316 - mae: 11.5947 - val_loss: 168.6587 - val_mae: 11.6051
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - loss: 192.0713 - mae: 11.9326 - val_loss: 163.4951 - val_mae: 11.3799
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - loss: 182.7928 - mae: 11.8292 - val_loss: 162.8072 - val_mae: 11.2517
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - loss: 180.1385 - mae: 11.5772 - val_loss: 164.8891 - val_mae: 11.5023
Epoch 9/20
40/40 ━━━━━━

In [8]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')
print(f'Test loss: {test_loss}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 809ms/step - loss: 164.3122 - mae: 11.4677
Test MAE: 11.522581100463867
Test loss: 162.76710510253906


In [9]:
for layer in base_model.layers[-10:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mean_squared_error', metrics=['mae'])
history_fine = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - loss: 257.1718 - mae: 13.2736 - val_loss: 162.1944 - val_mae: 11.4379
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 173.3695 - mae: 11.1365 - val_loss: 159.3848 - val_mae: 11.2408
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 163.2630 - mae: 10.7137 - val_loss: 162.2684 - val_mae: 11.4013
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 150.7781 - mae: 10.4245 - val_loss: 190.5565 - val_mae: 12.0645
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 152.9326 - mae: 10.4502 - val_loss: 195.9874 - val_mae: 12.1726
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 139.1940 - mae: 9.8755 - val_loss: 192.0729 - val_mae: 12.0722
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 129.7340 - mae: 9.6003 - val_loss: 191.1107 - val_mae: 12.0197
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 129.6863 - mae: 9.6471 - val_loss: 182.0372 - val_mae: 11.7610
Epoch 9/10
40/40 ━━━━━━━━━━

In [10]:
model.save("BetterSEOModel.h5")

In [11]:
new_model = tf.keras.models.load_model('BetterSEOModel.h5')

In [17]:
# Predict rating for a new screenshot
def predict_rating(screenshot):
    new_dimensions = (224,224)
    resized_image = cv2.resize(screenshot, new_dimensions, interpolation=cv2.INTER_AREA)
    screenshot = resized_image / 255.0
    screenshot = np.expand_dims(screenshot, axis=0)  
    predicted_rating = new_model.predict(screenshot)
    return predicted_rating[0][0]

# Load Screenshot and make prediction
new_screenshot = cv2.imread('../dataset/images/{0}'.format('animemojo.png'),cv2.IMREAD_COLOR)
predicted_rating = predict_rating(new_screenshot)
print(f'Predicted Rating: {predicted_rating}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted Rating: 72.94987487792969
